# GORILLA plotting tutorial

With the plotting option of GORILLA it is possible to compute and plot the orbit of a particle in a toroidal magnetic fusion device. For the visualization of the orbit, Poincare plots are used. The program also computes various constants of motion to check how the obtained results behave with respect to numerical accuracy.  

This tutorial guides through different input options of the code and shows a way how to execute the code with PYTHON. In principle one could change the repective options in the namelist-files like 'gorilla.inp' or 'tetra_grid.inp' per hand and then start the main executable 'test_gorilla-main.x'. However, here we will show how one can instead control these options dynamically in a PYTHON script using the the *f90nml*-namelist package.

***

## Preliminary: Using *f90nml* and *os* to alter namelists

First we load the *f90nml* package after installing it e.g. using pip. Additionally we load the default module *os* (this is used to execute shell commands in PYTHON such as getting the current working directory path, creating folders and starting the main executable of GORILLA).

In [1]:
import f90nml
import os

path_jupyter = os.getcwd()
print(path_jupyter)

/afs/itp.tugraz.at/user/grassl_g/MT/GORILLA/JUPYTER


The namelist `tetra_grid_nml` we want to look at in this example is located in the file `tetra_grid.inp`, which can be found in the *INPUT* folder. So we provide the correct path to that file and read it into a python dictionary. Make sure to set the *end_comma* property of the generated namelist-object to *True*.

In [2]:
filename= path_jupyter + '/../INPUT/tetra_grid.inp'
tetra_grid = f90nml.read(filename)
tetra_grid.end_comma = True
print(tetra_grid)

&tetra_grid_nml
    n1 = 100,
    n2 = 40,
    n3 = 40,
    grid_kind = 3,
    g_file_filename = 'MHD_EQUILIBRIA/g_file_for_test',
    convex_wall_filename = 'MHD_EQUILIBRIA/convex_wall_for_test.dat',
    netcdf_filename = 'MHD_EQUILIBRIA/netcdf_file_for_test.nc',
    boole_n_field_periods = .true.,
    n_field_periods_manual = 1,
    sfc_s_min = 0.1,
    theta_geom_flux = 1,
    theta0_at_xpoint = .true.,
    boole_write_mesh_obj = .false.,
    filename_mesh_rphiz = 'mesh_rphiz.obj',
    filename_mesh_sthetaphi = 'mesh_sthetaphi.obj',
/


You can now access and change the values of the listed *key:value pairs* in that dictonary by first indexing the wanted namelist and then the keyname:

In [3]:
print('The current number of grid points n1 is ' + str(tetra_grid['tetra_grid_nml']['n1']))

tetra_grid['tetra_grid_nml']['n1'] = 101

print('The new value of n1 is ' + str(tetra_grid['tetra_grid_nml']['n1']))

The current number of grid points n1 is 100
The new value of n1 is 101


After we have altered the desired options, we write our changed namelist to a new namelist-file (this is done to preserve the comments in the original file as well as the default settings). The final file should be located in the same folder as the main excutable `test_gorilla_main.x` so that it can then properly acess the file.

In [4]:
new_filename = path_jupyter + '/../tetra_grid.inp'
tetra_grid.write(new_filename, force = True);

The commments in the original `.inp` file explain the different options of the namelist `tetra_grid_nml`. We now leave the created input-file as it is and will overwrite it later when covering the input files in detail.

***

## Initialization

Assuming the proper packages (*f90nml* and *os*) are already loaded, the PYTHON code first needs to know where to find the used files. Therefore, it is necessary to specify some paths. 

For the here provided default values to work
* the folder *GORILLA* must not have been changed
* **the current working directory** has to be the folder of this JUPYTER notebook (check the print output of the next cell)
* and the code cells of this section have to be executed in order. 

**If in doubt, restart the notebook to clear the working variables and to reset the current path to this notebook.**
 
First, we get the current working directory:

In [5]:
path_script = os.getcwd()
#default: path_script= os.getcwd();
print(path_script)

/afs/itp.tugraz.at/user/grassl_g/MT/GORILLA/JUPYTER


The path of the main folder *GORILLA*:

In [6]:
path_main = path_script + '/../'
#default: path_main= path_script + '/../'

Next, the path where the FORTRAN code should be executed. Here we create a new clean RUN folder for that. Note that the explicit shell commands depend on the used operating system.

In [7]:
path_RUN = path_main + '/EXAMPLES/PYTHON_RUN/plot_poincare'
#default path_RUN= path_main + '/EXAMPLES/PYTHON_RUN/plot_poincare'

if not os.path.exists(path_RUN):
  os.makedirs(path_RUN)

os. chdir(path_RUN)
if os.listdir('../plot_poincare/'):
  os.system('rm -r ../plot_poincare/*')

In the *INPUT* folder the input files for the GORILLA code are already prepared. All input variables have a default value. These input files will be loaded into the PYTHON script later on.

In [8]:
path_inp_files = path_main + '/INPUT'
#default path_inp_files= path_main + '/INPUT';

The created files during this run will be stored in a seperated folder. Here we create it in the same directory as the JUPYTER notebook.

In [9]:
name_folder ='/data_plots/plot_poincare'
path_data_plots = path_script + name_folder

if not os.path.exists(path_data_plots):
  os.makedirs(path_data_plots)


***

## Loading the namelists

The *GORILLA* code requires some input files. All possible options for the code can be set in these files. In the INPUT folder, there are already default input files. In these files one can also find short explanations for ervery variable. In this script all input files will be loaded. The general way of working with the namelists in these input files using *f90nml* is explained in the preliminary at the top of this notebook.

In [10]:
gorilla = f90nml.read(path_inp_files + '/gorilla.inp')
gorilla.end_comma = True
tetra_grid = f90nml.read(path_inp_files + '/tetra_grid.inp')
tetra_grid.end_comma = True
gorilla_plot = f90nml.read(path_inp_files + '/gorilla_plot.inp')
gorilla_plot.end_comma = True


***

## Input file gorilla.inp

### Electrostatic potential

Change in the electrostatic potential within the plasma volume in Gaussian units 

In [11]:
gorilla['gorillanml']['eps_Phi'] = 0

### Coordinate system

1 ... ($R$,$\varphi$,$Z$) cylindrical coordinate system

2 ... ($s$,$\vartheta$,$\varphi$) symmetry flux coordinate system 

In [12]:
gorilla['gorillanml']['coord_system'] = 2

### Particle species

1 ... electron

2 ... deuterium ion

3 ... alpha particle

In [13]:
gorilla['gorillanml']['ispecies'] = 1

### Periodic coordinate particle re-location

*True* ... Particles are re-located at initialization in the case of a periodic coordinate, if they are outside the computation domain.

*False* .. Particles are not re-located at initialization (This might lead to error if particles are outside the computation domain)

In [14]:
gorilla['gorillanml']['boole_periodic_relocation'] = True

### GORILLA pusher options

1 ... numerical RK pusher

2 ... polynomial pusher

In [15]:
gorilla['gorillanml']['ipusher'] = 2

### Numerical pusher options

##### accuracy for integration step

*False* ... RK4

*True* ... adaptive ODE45

In [16]:
gorilla['gorillanml']['boole_pusher_ode45'] = False

##### ODE45 relative accuracy (pusher orbit)

In [17]:
gorilla['gorillanml']['rel_err_ode45'] = 1e-8

##### Physical time - orbit parameter relation

*False* ... $dt/d\tau$ is a linear function of position in each tetrahedron

*True* ... $dt/d\tau$ is a constant averaged quantity in each tetrahedron

In [18]:
gorilla['gorillanml']['boole_dt_tau'] = True

##### Precomputation for Newton iterations

*False* ... Compute coefficients for 2nd order polynomial solution (Newton velocity and acceleration) in each tetrahedron ($dzd\tau$) without precomputation

*True* ... Use precomputation of coefficients for 2nd order polynomial solution (Newton velocity and acceleration)

In [19]:
gorilla['gorillanml']['boole_newton_precalc'] = False

### Polynomial pusher options

##### Polynomial order for orbit pusher

In [20]:
gorilla['gorillanml']['poly_order'] = 4

##### Settings for precomputation of coefficients

0 ... No precomputation: All coefficients are computed on the fly (power of matrices)

1 ... Precomputation of coefficients and factorization of powers of $v_{\perp}$

2 ... Same as 1 but with further precomputation of operatorb in $b$

3 ... Extended precomputation of coefficients for constant $v_{\perp}$

(Precomputation is done for each tetrahedron for constant perpinv) NOT IMPLEMENTED YET

In [21]:
gorilla['gorillanml']['i_precomp'] = 0

##### Face prediction with 2nd order polynomial

*True* ... Face guessing algorithm is used

*False* ... NOT used

In [22]:
gorilla['gorillanml']['boole_guess'] = True

### Processing of particle handover to tetrahedron neighbour

1 ... Processing with special treatment of periodic boundaries and manipulation of periodic position values

2 ... Position exchange via Cartesian variables (skew_coordinates) - Necessary precomputation is included

In [23]:
gorilla['gorillanml']['handover_processing_kind'] = 1

### Noise on axisymmetric electromagnetic field (Tokamak)

##### Add axisymmetric noise to vector potential $A_k$

*True* ... Add axisymmetric random noise ($\xi$ = 0 ... 1) to the co-variant component of the vector potential $A_k$

$A_k' = A_k(1 + \epsilon{}_A\xi)$

*False* ... Add no axisymmetric noise to $A_k$

In [24]:
gorilla['gorillanml']['boole_axi_noise_vector_pot'] = False

##### Relative Magnitude of axisymmetric random noise $\epsilon{}_A$ of vector potential

In [25]:
gorilla['gorillanml']['axi_noise_eps_A'] = 1e-1

##### Add axisymmetric noise to electrostatic potential $\phi$

*True* ... Add axisymmetric random noise ($\xi$ = 0 ... 1) to the electrostatic potential $\phi$

$\phi' = \phi(1 + \epsilon{}_{\phi}\xi)$

*False* ... Add no axisymmetric noise to $\phi$

In [26]:
gorilla['gorillanml']['boole_axi_noise_elec_pot'] = False

##### Relative Magnitude of axisymmetric random noise $\epsilon{}_{\phi}$ of electrostatic potential

In [27]:
gorilla['gorillanml']['axi_noise_eps_Phi'] = 3e-1

##### Add non-axisymmetric noise to vector potential $A_k$

In [28]:
gorilla['gorillanml']['boole_non_axi_noise_vector_pot'] = False

##### Relative Magnitude of non-axisymmetric random noise $\epsilon{}_A$ of vector potential

In [29]:
gorilla['gorillanml']['non_axi_noise_eps_A'] = 1e-4

### Harmonic perturbation on axisymmetric electromagnetic field

##### Add helical harmonic perturbation of $A_{\varphi}$

*False* ... no perturbation

*True* ... helical perbutation on:

$A_{\varphi}' = A_{\varphi} + A_{\varphi}* \epsilon{}_{A_{\varphi}}^{helical} * \cos{(m_{fourier}*\vartheta +n_{fourier}*\varphi)}$

In [30]:
gorilla['gorillanml']['boole_helical_pert'] = False

##### Amplitude of helical harmonic perturbation $\epsilon{}_{A_{\varphi}}^{helical}$

In [31]:
gorilla['gorillanml']['helical_pert_eps_Aphi'] = 1e-1

##### Fourier modes ($m_{fourier}$, $n_{fourier}$) of helical perturbation 

In [32]:
gorilla['gorillanml']['helical_pert_m_fourier'] = 2
gorilla['gorillanml']['helical_pert_n_fourier'] = 2


***

## Input file tetra_grid.inp

### Grid size

Rectangular: $n_R$, Field-aligned: $n_s$

In [33]:
tetra_grid['tetra_grid_nml']['n1'] = 100

Rectangular: $n_{\varphi}$, Field-aligned: $n_{\varphi}$ 

In [34]:
tetra_grid['tetra_grid_nml']['n2'] = 40

Rectangular: $n_Z$, Field-aligned: $n_{\vartheta}$

In [35]:
tetra_grid['tetra_grid_nml']['n3'] = 40

### Grid kind

1 ... rectangular grid for axisymmetric EFIT data (g-file)

2 ... field-aligned grid for axisymmetric EFIT data (g-file)

3 ... field-aligned grid for non-axisymmetric VMEC (netcdf)

In [36]:
tetra_grid['tetra_grid_nml']['grid_kind'] = 3


### Source files of fields

GORILLA needs the electromagnetic fields as input to perform the trajectory integration. Additionally, the fields are also needed to generate the field-aligned grid or to perform the triangle classification when dealing with the WEST geometry. The location (default is the folder *MHD_EQUILIBRIA*) and names of the corresponding MHD equilibrium files are specified here:

In [37]:
tetra_grid['tetra_grid_nml']['g_file_filename'] = 'MHD_EQUILIBRIA/g_file_for_test' 
tetra_grid['tetra_grid_nml']['convex_wall_filename'] = 'MHD_EQUILIBRIA/convex_wall_for_test.dat' 
tetra_grid['tetra_grid_nml']['netcdf_filename'] = 'MHD_EQUILIBRIA/netcdf_file_for_test.nc' 

### Automatic determination of field period

*True* ... number of field periods is selected automatically (Tokamak = 1,       Stellarator depending on VMEC equilibrium)

*False* ... number of field periods is selected manually (see below)

In [38]:
tetra_grid['tetra_grid_nml']['boole_n_field_periods'] = True

### Number of field periods (manual)

In [39]:
tetra_grid['tetra_grid_nml']['n_field_periods_manual'] = 1

### Symmetry Flux Coordinates Annulus

During the generation of the field aligned grid with Symmetry Flux Coodinates, a small region around the magnetic axis is left out, due to insufficient precission of conversion functions for this region. The grid is therefore only generated down to a minimal value of the flux fluxcoordinate $s$.

In [40]:
tetra_grid['tetra_grid_nml']['sfc_s_min'] = 0.1

### Option for poloidal angle-variable

By default the aligned grid is generated by having equidistant points in SFC ($s$,$\vartheta$,$\phi$). However, for the flux poloidal angle $\vartheta$ there is the option to have the points equidistant in terms of the geometrical poloidal angle $\theta$ instead. This avoids undesired deformations of the grid in real space.

1 ... angle scaling in symmetry flux coordinates $\vartheta$

2 ... angle scaling in geometrical poloidal angle $\theta$

In [41]:
tetra_grid['tetra_grid_nml']['theta_geom_flux'] = 1

### Poloidal angle-variable origin 

*True* ... angle-variable starts at the line between O- and X-Point

*False* ... angle-variable starts at the line between O-Point and intersection between O-Point-[1,0]-straight-line and separatrix

In [42]:
tetra_grid['tetra_grid_nml']['theta0_at_xpoint'] = True


### Save generated grid as object files

*GORILLA* generates the mesh (vertex positions plus assignment to the tetrahedra) everytime the executable is called. It therefore only exists temporary during runtime. However, one can set it to be saved in an object file.

In [43]:
tetra_grid['tetra_grid_nml']['boole_write_mesh_obj'] = False

### Filename for mesh object file in cylindrical coordinates

In [44]:
tetra_grid['tetra_grid_nml']['filename_mesh_rphiz'] = 'mesh_rphiz.obj' 

### Filename for mesh object file in symmetry flux coordintes 

In [45]:
tetra_grid['tetra_grid_nml']['filename_mesh_sthetaphi'] = 'mesh_sthetaphi.obj' 

***

## Input file gorilla_plot.inp

### Switch for starting position options
1 ... Single orbit - Starting positions and pitch for the orbit are taken from file (see below) [First Line]

2 ... Single orbit - Starting positions and pitch for the orbit are taken from starting drift surfaces (see below)

3 ... Multiple orbits - Starting positions and pitch for orbits are taken from file (see below) [Every Line New Starting position]

4 ... Multiple orbits - Starting positions and pitch for orbits are taken from drift surfaces with regular spacing (see below)

In [46]:
gorilla_plot['gorilla_plot_nml']['i_orbit_options'] = 2

### Total individual orbit flight time for plotting

In [47]:
gorilla_plot['gorilla_plot_nml']['total_orbit_time'] = 2e-1

### Total energy of particle in eV

In [48]:
gorilla_plot['gorilla_plot_nml']['energy_eV_start'] = 3e3

### Plot Poincaré cuts ($\varphi = 0$)

##### Switch for plotting Poincaré cuts ($\varphi = 0$)

In [49]:
gorilla_plot['gorilla_plot_nml']['boole_poincare_phi_0'] = True

##### Number of skipped (non-printed) Poincaré cuts ($\varphi = 0$)

In [50]:
gorilla_plot['gorilla_plot_nml']['n_skip_phi_0'] = 10

##### Filename for Poincaré cuts ($\varphi = 0$) in cylindrical coordinates ($R$,$\varphi$,$Z$)

In [51]:
gorilla_plot['gorilla_plot_nml']['filename_poincare_phi_0_rphiz'] = 'poincare_plot_phi_0_rphiz.dat'

##### Filename for Poincaré cuts ($\varphi = 0$) in symmetry flux coordinates ($s$,$\vartheta$,$\varphi$)

In [52]:
gorilla_plot['gorilla_plot_nml']['filename_poincare_phi_0_sthetaphi'] = 'poincare_plot_phi_0_sthetaphi.dat'

### Plot Poincaré cuts ($v_{\parallel}$ = 0)

***

Finally, we again save all our changes into the `.inp` file which will be used by *GORILLA*.

In [78]:
tetra_grid.write(new_filename, force= True);